In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_columns',0)

/tmp/ipykernel_14659/3884596303.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv('yellow_tripdata_2023-01.csv',low_memory=False)

In [3]:
after_filtered_df=df[df['VendorID'] != 'VendorID']

In [4]:
after_filtered_df['VendorID']=after_filtered_df['VendorID'].astype('category')
after_filtered_df['tpep_pickup_datetime']=pd.to_datetime(after_filtered_df['tpep_pickup_datetime'])
after_filtered_df['tpep_dropoff_datetime']=pd.to_datetime(after_filtered_df['tpep_dropoff_datetime'])
after_filtered_df['passenger_count']=pd.to_numeric(after_filtered_df['passenger_count'],downcast='integer')
after_filtered_df['trip_distance']=pd.to_numeric(after_filtered_df['trip_distance'])
after_filtered_df['RatecodeID']=after_filtered_df['RatecodeID'].astype('category')
after_filtered_df['store_and_fwd_flag']=after_filtered_df['store_and_fwd_flag'].astype('category')
after_filtered_df['PULocationID']=pd.to_numeric(after_filtered_df['PULocationID'])
after_filtered_df['DOLocationID']=pd.to_numeric(after_filtered_df['DOLocationID'])
after_filtered_df['payment_type']=after_filtered_df['payment_type'].astype('category')
after_filtered_df['fare_amount']=pd.to_numeric(after_filtered_df['fare_amount'])
after_filtered_df['extra']=pd.to_numeric(after_filtered_df['extra'])
after_filtered_df['mta_tax']=pd.to_numeric(after_filtered_df['mta_tax'])
after_filtered_df['tip_amount']=pd.to_numeric(after_filtered_df['tip_amount'])
after_filtered_df['tolls_amount']=pd.to_numeric(after_filtered_df['tolls_amount'])
after_filtered_df['improvement_surcharge']=pd.to_numeric(after_filtered_df['improvement_surcharge'])
after_filtered_df['total_amount']=pd.to_numeric(after_filtered_df['total_amount'])
after_filtered_df['congestion_surcharge']=pd.to_numeric(after_filtered_df['congestion_surcharge'])
after_filtered_df['airport_fee']=pd.to_numeric(after_filtered_df['airport_fee'])

In [5]:
after_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3066766 entries, 0 to 3066811
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               category      
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             category      
 6   store_and_fwd_flag     category      
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           category      
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [6]:
import psycopg2
import pandas as pd
from psycopg2 import extras

try:
    # Establishing the connection
    conn = psycopg2.connect(
        database="postgres", user='postgres', password='password', host='127.0.0.1', port='5432')
    
    # Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    
    # Create table yellow_tripdata_2023_01 if not exists
    cursor.execute('''CREATE TABLE IF NOT EXISTS yellow_tripdata_2023_01_test (
                            id SERIAL PRIMARY KEY,
                            VendorID int, 
                            tpep_pickup_datetime TEXT,
                            tpep_dropoff_datetime TEXT,
                            passenger_count int,
                            trip_distance float,
                            RatecodeID int,
                            store_and_fwd_flag int,
                            PULocationID int,
                            DOLocationID int,
                            payment_type int,
                            fare_amount float,
                            extra float,
                            mta_tax float,
                            tip_amount float,
                            tolls_amount float,
                            improvement_surcharge float,
                            total_amount float,
                            congestion_surcharge float,
                            airport_fee float);''')
    
    # Commit the table creation
    conn.commit()
    print("Table 'yellow_tripdata_2023_01_test' created successfully")
    
    # Prepare data as a list of tuples
    data = [tuple(row) for row in df.itertuples(index=False, name=None)]
    
    # Construct the INSERT INTO statement
    insert_query = """
        INSERT INTO yellow_tripdata_2023_01_test (VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, 
        trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, 
        mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee)
        VALUES %s
    """
    
    # Execute the INSERT INTO statement using execute_values()
    extras.execute_values(cursor, insert_query, data, page_size=100000) #page_size is number of rows to be inserted  
    
    # Committing the transaction
    conn.commit()
    print("Data inserted successfully in yellow_tripdata_2023_01_test table")
    
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL or inserting data:", error)

finally:
    # Closing the connection
    if conn:
        cursor.close()
        conn.close()
        print("PostgreSQL connection is closed")

Table 'yellow_tripdata_2023_01_test' created successfully
Error while connecting to PostgreSQL or inserting data: invalid input syntax for type integer: "1.0"
LINE 5: ... ('2','2023-01-01 00:32:10','2023-01-01 00:40:36','1.0','0.9...
                                                             ^

PostgreSQL connection is closed


In [1]:
import pandas as pd
def test_func():
    rate_dim_data={'rate_id':[1,2,3,4,5,6],
                  'rate_description':["Standard rate","JFK","Newark","Nassau or Westchester",
                                         "Negotiated fare","Group ride"]
                    }
    rate_dim=pd.DataFrame(rate_dim_data)


    #prepare data for store_and_forward_dim table
    store_and_forward_dim_data={'store_and_forward_id':[1,2],
                  'store_and_forward_flag':["Y","N"],
                  'store_and_forward_description':["store and forward trip","not a store and forward trip"]
                    }
    store_and_forward_dim=pd.DataFrame(store_and_forward_dim_data)

    #prepare data for vendor_dim table
    after_duplicate_filtered_df_data={
                    "Y":1,
                    "N":2}
    data_frame_dict={
        'payment_dim':payment_dim,
        'rate_dim':rate_dim,
        'store_and_forward_dim':store_and_forward_dim
    }
    
    return data_frame_dict


In [2]:
for df,name in data_frame_dict.items():
    print(df)

NameError: name 'data_frame_dict' is not defined